In [362]:
import os
from pathlib import Path
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from dotenv import load_dotenv
load_dotenv()

DEFAULE_OUTPUT_DIR = Path(r"./embed_documents")
# DEFAULE_OUTPUT_DIR = r"./embed_documents"

document_mk_file = "../finish/VIVA(2022.02).md"
car_model = Path(document_mk_file).stem.split("(")[0]

# OUTPUT_DIR = DEFAULE_OUTPUT_DIR / car_model.upper()
OUTPUT_DIR = DEFAULE_OUTPUT_DIR


with open(document_mk_file, "r", encoding="utf-8") as f:
    markdown_document = f.read()

headers_to_split_on = [
    ("#", "#"),
    ("##", "##"),
    ("###", "###"),
    ("####", "####"),
    ("#####", "#####"),
]

In [363]:
# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

len(md_header_splits), md_header_splits[:5]

(100,
 [Document(page_content='# Gogoro Viva  \n發佈⽇期:2022.02.16 適⽤⾞款:\nVIVA Lite VIVA VIVA Plus VIVA BASIC VIVA KEYLESS\niQ System® 版本:6.5 Gogoro® App 版本:2.19 以後 製造商:睿能創意股份有限公司 地址:桃園市⿔⼭區頂湖路 33 號 電話:03-273-0900 客服中⼼電話:0800-365-996 Copyright© 2021 睿能創意股份有限公司 著作權所有,並保留⼀切權利。本⾞主⼿冊中任何部分未經睿能創意股份有限公司事 前書⾯同意,不得以任何形式轉載、複製或拷⾙。', metadata={'#': 'Gogoro Viva'}),
  Document(page_content='## 目錄\n| ⽬                                                                   |     |\n|----------------------------------------------------------------------|-----|\n| 錄                                                                   |     |\n| 1. 在 您 上 路 之 前                                                 | 5   |\n| 1.1 如 何 使 ⽤ 本 ⼿ 冊                                             | 5   |\n| 1.2 安 全 提 醒                                                      | 5   |\n| 1.2.1 每 次 騎 乘 前 應 檢 查 項 ⽬                                  | 6   |\n| 1.3 G o g o r o S m a r t s c o o t e r ® 簡 介                      | 8   |\n| 1.3.1

In [364]:
import re

def extract_page_metadata(text):
    page_metadata = None
    cleaned_lines = []
    page_pattern = re.compile(r'^###### (\d+)')
    header_pattern = re.compile(r'^(#{1,5}) ')
    # header_pattern = re.compile(r'^# ')

    for line in text.split('\n'):
        match = page_pattern.match(line)
        if match:
            page_metadata = int(match.group(1))
        elif not header_pattern.match(line):
            cleaned_lines.append(line)

    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text, page_metadata

_content, _page_metadata = extract_page_metadata(md_header_splits[12].page_content)
_content, _page_metadata

('![13_image_0.png](13_image_0.png)  \n- 除了隨⾞標配的 iQ System® 智慧鑰匙卡,您尚可⾃費加購 Gogoro Smart Coin,讓感 應鑰匙成為時尚配件,隨⾝輕鬆配戴。獨特感應扣 Smart Coin,僅有硬幣⼤⼩,還能 與專屬矽膠⼿環或掛環搭配,完美融入各種⽇常情境,降低感應鑰匙遺留於⾞廂內的 風險。  \n- 使⽤⽅式及注意事項與 iQ System® 智慧鑰匙卡相同。',
 14)

In [365]:
md_header_splits[12].metadata

{'#': 'Gogoro Viva',
 '##': '1. 在您上路之前',
 '###': '1.3 Gogoro Smartscooter® 簡介',
 '####': '1.3.2 開關機鑰匙',
 '#####': '1.3.2.4 Gogoro Smart Coin'}

In [366]:
from langchain.schema import Document

def clean_context(text):
    text = re.sub(r'\s{2,}', '  ', text)  # Replace multiple spaces with two spaces
    text = text.replace("**", "")  # Remove double asterisks
    return text

# Function to replace image paths in the content
def replace_image_paths(text):
    new_text = re.sub(r'!\[(.*?)\]\((.*?)\)', rf'![\1](app/static/{car_model}/\2)', text)
    return new_text

processed_splits = []
for doc in md_header_splits:
    content, page_metadata = extract_page_metadata(doc.page_content)

    content = replace_image_paths(content)

    cleaned_content = clean_context(content)

    new_metadata = doc.metadata.copy()
    header = "\n".join(doc.metadata)
    # new_metadata['car_model'] = car_model
    if page_metadata:
        new_metadata['page'] = page_metadata

    processed_splits.append(Document(page_content=cleaned_content, metadata=new_metadata))
len(processed_splits), processed_splits[:5]

(100,
 [Document(page_content='發佈⽇期:2022.02.16 適⽤⾞款:\nVIVA Lite VIVA VIVA Plus VIVA BASIC VIVA KEYLESS\niQ System® 版本:6.5 Gogoro® App 版本:2.19 以後 製造商:睿能創意股份有限公司 地址:桃園市⿔⼭區頂湖路 33 號 電話:03-273-0900 客服中⼼電話:0800-365-996 Copyright© 2021 睿能創意股份有限公司 著作權所有,並保留⼀切權利。本⾞主⼿冊中任何部分未經睿能創意股份有限公司事 前書⾯同意,不得以任何形式轉載、複製或拷⾙。', metadata={'#': 'Gogoro Viva'}),
  Document(page_content='| ⽬  |  |\n|----------------------------------------------------------------------|-----|\n| 錄  |  |\n| 1. 在 您 上 路 之 前  | 5  |\n| 1.1 如 何 使 ⽤ 本 ⼿ 冊  | 5  |\n| 1.2 安 全 提 醒  | 5  |\n| 1.2.1 每 次 騎 乘 前 應 檢 查 項 ⽬  | 6  |\n| 1.3 G o g o r o S m a r t s c o o t e r ® 簡 介  | 8  |\n| 1.3.1 操 作 流 程 概 述  | 8  |\n| 1.3.2 開 關 機 鑰 匙  | 9  |\n| 1.3.2.1 機 械 式 鑰 匙  | 1 0 |\n| 1.3.2.2 iQ S y s t e m ® 無 線 智 慧 鑰 匙  | 1 1 |\n| 1.3.2.3 iQ S y s t e m ® 智 慧 鑰 匙 卡  | 1 2 |\n| 1.3.2.4 G o g o r o S m a r t C oin  | 1 4 |\n| 1.3.2.5 以 ⼿ 機 作 為 鑰 匙  | 1 5 |\n| 1.3.3 啟 動 及 關 閉 ⾺ 達  | 1 6 |\n| 1.3.3.1 啟 動 ⾺ 達  | 1 6 |\n| 1.3.3.2 關 閉 ⾺ 達  | 1 6 |\n| 1.4 G o g o 

In [367]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 500
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(processed_splits)
len(splits), splits[:5]

(207,
 [Document(page_content='發佈⽇期:2022.02.16 適⽤⾞款:\nVIVA Lite VIVA VIVA Plus VIVA BASIC VIVA KEYLESS\niQ System® 版本:6.5 Gogoro® App 版本:2.19 以後 製造商:睿能創意股份有限公司 地址:桃園市⿔⼭區頂湖路 33 號 電話:03-273-0900 客服中⼼電話:0800-365-996 Copyright© 2021 睿能創意股份有限公司 著作權所有,並保留⼀切權利。本⾞主⼿冊中任何部分未經睿能創意股份有限公司事 前書⾯同意,不得以任何形式轉載、複製或拷⾙。', metadata={'#': 'Gogoro Viva'}),
  Document(page_content='| ⽬  |  |\n|----------------------------------------------------------------------|-----|\n| 錄  |  |\n| 1. 在 您 上 路 之 前  | 5  |\n| 1.1 如 何 使 ⽤ 本 ⼿ 冊  | 5  |\n| 1.2 安 全 提 醒  | 5  |\n| 1.2.1 每 次 騎 乘 前 應 檢 查 項 ⽬  | 6  |\n| 1.3 G o g o r o S m a r t s c o o t e r ® 簡 介  | 8  |\n| 1.3.1 操 作 流 程 概 述  | 8  |\n| 1.3.2 開 關 機 鑰 匙  | 9  |\n| 1.3.2.1 機 械 式 鑰 匙  | 1 0 |\n| 1.3.2.2 iQ S y s t e m ® 無 線 智 慧 鑰 匙  | 1 1 |\n| 1.3.2.3 iQ S y s t e m ® 智 慧 鑰 匙 卡  | 1 2 |\n| 1.3.2.4 G o g o r o S m a r t C oin  | 1 4 |', metadata={'#': 'Gogoro Viva', '##': '目錄'}),
  Document(page_content='| 1.3.2.3 iQ S y s t e m ® 智 慧 鑰 匙 卡  | 1 2 |\n| 1.3.2.4 G o g o r

In [368]:
max([len(split.page_content) for split in splits])

500

In [369]:
c_splits = []
for split in splits:
    new_metadata = split.metadata.copy()
    new_metadata["car_model"] = car_model.upper()
    new_content = ''
    for key in sorted(split.metadata.keys()):
        if key.startswith('#') and key != '#':
            clear_header_metadata = split.metadata[key].replace("**", "")
            new_content += f"{key} {clear_header_metadata}\n"
    c_splits.append(Document(page_content=new_content + split.page_content, metadata=new_metadata))

len(c_splits), c_splits[:5]

(207,
 [Document(page_content='發佈⽇期:2022.02.16 適⽤⾞款:\nVIVA Lite VIVA VIVA Plus VIVA BASIC VIVA KEYLESS\niQ System® 版本:6.5 Gogoro® App 版本:2.19 以後 製造商:睿能創意股份有限公司 地址:桃園市⿔⼭區頂湖路 33 號 電話:03-273-0900 客服中⼼電話:0800-365-996 Copyright© 2021 睿能創意股份有限公司 著作權所有,並保留⼀切權利。本⾞主⼿冊中任何部分未經睿能創意股份有限公司事 前書⾯同意,不得以任何形式轉載、複製或拷⾙。', metadata={'#': 'Gogoro Viva', 'car_model': 'VIVA'}),
  Document(page_content='## 目錄\n| ⽬  |  |\n|----------------------------------------------------------------------|-----|\n| 錄  |  |\n| 1. 在 您 上 路 之 前  | 5  |\n| 1.1 如 何 使 ⽤ 本 ⼿ 冊  | 5  |\n| 1.2 安 全 提 醒  | 5  |\n| 1.2.1 每 次 騎 乘 前 應 檢 查 項 ⽬  | 6  |\n| 1.3 G o g o r o S m a r t s c o o t e r ® 簡 介  | 8  |\n| 1.3.1 操 作 流 程 概 述  | 8  |\n| 1.3.2 開 關 機 鑰 匙  | 9  |\n| 1.3.2.1 機 械 式 鑰 匙  | 1 0 |\n| 1.3.2.2 iQ S y s t e m ® 無 線 智 慧 鑰 匙  | 1 1 |\n| 1.3.2.3 iQ S y s t e m ® 智 慧 鑰 匙 卡  | 1 2 |\n| 1.3.2.4 G o g o r o S m a r t C oin  | 1 4 |', metadata={'#': 'Gogoro Viva', '##': '目錄', 'car_model': 'VIVA'}),
  Document(page_content='## 目錄\n| 1.3.2.3 

In [370]:
embeddings = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")
db = FAISS.from_documents(c_splits, embeddings)
db.index.ntotal

207

In [371]:
query = "如何開啟和關閉 Gogoro 電動機車的系統電源？"
retriever = db.as_retriever()
docs = retriever.invoke(query)

docs

[Document(page_content='## 3. 準備上路\n### 3.2 無線鑰匙⾞種\n無線鑰匙⾞種可以使⽤包括 iQ System® 無線智慧鑰匙(圓形按鍵式)、 iQ System® 智慧鑰 匙卡(卡片式)以及 Gogoro® App 等非接觸式的開、關機⽅式。(依⾞種年式可能略有差 異)\n每次關閉系統電源前,務必確認鑰匙、⼿機未放置於座墊下置物箱,以免意外把鑰匙、⼿機鎖 在裡⾯,導致無法再次開啟系統電源。  請隨時關閉座墊下置物箱,以免因長時間未關閉座墊,導致 Gogoro Network® 智慧電池無法 對備⽤電池充電,進⽽導致備⽤電池過度放電⽽損壞。  無線鑰匙⾞種,在「⾺達已啟動」狀態時,無法關閉系統電源。 請先將⾺達關閉,才能關閉系統電源。\n無線鑰匙⾞種設有頭燈延遲熄滅功能,部分⾞種可使⽤ Gogoro® App 設定關閉系統電源後後 頭燈熄滅之時間長短。  每次騎乘前,請先轉動龍頭⾄最左邊再回正,確保龍頭鎖已經正確解鎖,龍頭可以⾃由轉向, 再開始騎乘。  無線鑰匙⾞種,系統預設在電源關閉時並不會鎖上龍頭。  若您想要每次關閉系統電源時都⾃動鎖上龍頭,可以在 Gogoro® App 中啟⽤「⾃動龍頭鎖」', metadata={'#': 'Gogoro Viva', '##': '3. 準備上路', '###': '3.2 無線鑰匙⾞種', 'page': 33, 'car_model': 'VIVA'}),
 Document(page_content='## 1. 在您上路之前\n### 1.3 Gogoro Smartscooter® 簡介\n#### 1.3.3 啟動及關閉⾺達\n系統電源開啟後,必須先啟動⾺達,才能開始騎乘。⽽在結束騎乘後,也必須先關閉⾺達,才 能關閉系統電源並上鎖。( 參閱「4.3 啟動及關閉⾺達」)', metadata={'#': 'Gogoro Viva', '##': '1. 在您上路之前', '###': '1.3 Gogoro Smartscooter® 簡介', '####': '1.3.3 啟動及關閉⾺達', 'page': 16, 'car_model': 'VIVA'}),
 Document(page_content='## 3. 準備上路\n### 3.2 無線鑰匙⾞種\n####

In [372]:
db.similarity_search_with_score(query, top_k=5)

[(Document(page_content='## 3. 準備上路\n### 3.2 無線鑰匙⾞種\n無線鑰匙⾞種可以使⽤包括 iQ System® 無線智慧鑰匙(圓形按鍵式)、 iQ System® 智慧鑰 匙卡(卡片式)以及 Gogoro® App 等非接觸式的開、關機⽅式。(依⾞種年式可能略有差 異)\n每次關閉系統電源前,務必確認鑰匙、⼿機未放置於座墊下置物箱,以免意外把鑰匙、⼿機鎖 在裡⾯,導致無法再次開啟系統電源。  請隨時關閉座墊下置物箱,以免因長時間未關閉座墊,導致 Gogoro Network® 智慧電池無法 對備⽤電池充電,進⽽導致備⽤電池過度放電⽽損壞。  無線鑰匙⾞種,在「⾺達已啟動」狀態時,無法關閉系統電源。 請先將⾺達關閉,才能關閉系統電源。\n無線鑰匙⾞種設有頭燈延遲熄滅功能,部分⾞種可使⽤ Gogoro® App 設定關閉系統電源後後 頭燈熄滅之時間長短。  每次騎乘前,請先轉動龍頭⾄最左邊再回正,確保龍頭鎖已經正確解鎖,龍頭可以⾃由轉向, 再開始騎乘。  無線鑰匙⾞種,系統預設在電源關閉時並不會鎖上龍頭。  若您想要每次關閉系統電源時都⾃動鎖上龍頭,可以在 Gogoro® App 中啟⽤「⾃動龍頭鎖」', metadata={'#': 'Gogoro Viva', '##': '3. 準備上路', '###': '3.2 無線鑰匙⾞種', 'page': 33, 'car_model': 'VIVA'}),
  0.53079337),
 (Document(page_content='## 1. 在您上路之前\n### 1.3 Gogoro Smartscooter® 簡介\n#### 1.3.3 啟動及關閉⾺達\n系統電源開啟後,必須先啟動⾺達,才能開始騎乘。⽽在結束騎乘後,也必須先關閉⾺達,才 能關閉系統電源並上鎖。( 參閱「4.3 啟動及關閉⾺達」)', metadata={'#': 'Gogoro Viva', '##': '1. 在您上路之前', '###': '1.3 Gogoro Smartscooter® 簡介', '####': '1.3.3 啟動及關閉⾺達', 'page': 16, 'car_model': 'VIVA'}),
  0.5360478),
 (Document(page_content='

# Saving and Loading

In [373]:
db.save_local(OUTPUT_DIR / car_model.upper())

In [17]:
from typing import List, Union
from tqdm import tqdm

def index_docs(splitter, documents: List[str], dest_embed_dir) -> None:
    """
    Split the document into chunks and create embeddings for the document

    Args:
        url: Source url for the document.
        splitter: Splitter used to split the document
        documents: list of documents whose embeddings needs to be created
        dest_embed_dir: destination directory for embeddings

    Returns:
        None
    """
    embeddings = NVIDIAEmbeddings(model="nvolveqa_40k")
    
    for document in tqdm(documents):
        texts = splitter.split_text(document.page_content)

        # metadata to attach to document
        metadatas = [document.metadata]

        # create embeddings and add to vector store
        if os.path.exists(dest_embed_dir):
            update = FAISS.load_local(folder_path=dest_embed_dir, embeddings=embeddings, allow_dangerous_deserialization=True)
            update.add_texts(texts, metadatas=metadatas)
            update.save_local(folder_path=dest_embed_dir)
        else:
            docsearch = FAISS.from_texts(texts, embedding=embeddings, metadatas=metadatas, allow_dangerous_deserialization=True)
            docsearch.save_local(folder_path=dest_embed_dir)

index_docs(car_model, text_splitter, splits, OUTPUT_DIR)

c:\Users\BR\AppData\Local\miniconda3\envs\NM\lib\site-packages\langchain_nvidia_ai_endpoints\_statics.py:361: UserWarning: Model nvolveqa_40k is deprecated. Using NV-Embed-QA instead.
  warnings.warn(
c:\Users\BR\AppData\Local\miniconda3\envs\NM\lib\site-packages\langchain_nvidia_ai_endpoints\embeddings.py:87: UserWarning: Setting truncate="END" for nvolveqa_40k backward compatibility
  warnings.warn(


## Merge

In [11]:
if os.path.exists(OUTPUT_DIR):
    _db = FAISS.load_local(folder_path = OUTPUT_DIR, embeddings=embeddings, allow_dangerous_deserialization=True)
    _db.merge_from(db)
    _db.save_local(OUTPUT_DIR)
else:
    db.save_local(OUTPUT_DIR)

In [6]:
OUTPUT_DIR

'./embedding_output'

In [ ]:
db = FAISS.from_documents(docs, embeddings)
print(db.index.ntotal)

In [374]:
import glob

embeddings = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")
folders = [folder for folder in glob.glob("embed_documents/*") if folder != "embed_documents/ALL"]

main_db = FAISS.load_local(folder_path=folders[0], embeddings=embeddings, allow_dangerous_deserialization=True)

# 遍歷所有資料夾並合併索引
for folder in folders[1:]:
    db = FAISS.load_local(folder_path=folder, embeddings=embeddings, allow_dangerous_deserialization=True)
    main_db.merge_from(db)

# 將合併後的索引儲存到本地
main_db.save_local(OUTPUT_DIR / "ALL")

In [156]:
print(main_db.index.ntotal)

599


In [157]:
# query = "如何開啟和關閉 Gogoro 電動機車的系統電源？"
query = "智慧鑰匙卡感應器的位置在哪裡？"
retriever = main_db.as_retriever(search_kwargs={"k": 4, "filter": {"car_model": "CROSSOVER"}})
docs = retriever.invoke(query)

docs

[Document(page_content='左方視角\niQ System® 智慧鑰匙卡感應器位置\n「iQ System® 智慧鑰匙卡感應器」僅配備於使用無線鑰匙之車種。\n![22_image_0.png](app/static/CrossOver/22_image_0.png)', metadata={'#': 'Gogoro Crossover 系列 Smartscooter® **智慧電動機車** 使用手冊', '##': '2.1 Crossover 各部位名稱', 'page': 22, 'car_model': 'CROSSOVER'}),
 Document(page_content='iQ System® 智慧鑰匙卡感應器的位置依車種不同,見下圖 :  ![37_image_0.png](app/static/CrossOver/37_image_0.png)  ![37_image_1.png](app/static/CrossOver/37_image_1.png)  ![37_image_2.png](app/static/CrossOver/37_image_2.png)  ![37_image_3.png](app/static/CrossOver/37_image_3.png)', metadata={'#': 'Gogoro Crossover 系列 Smartscooter® **智慧電動機車** 使用手冊', '##': '3. **準備上路**', '###': '3.2 **無線鑰匙車種**', '####': '3.2.2iQ System® 智慧鑰匙卡', '#####': '3.2.2.1 iQ System® 智慧鑰匙卡感應器位置', 'page': 38, 'car_model': 'CROSSOVER'}),
 Document(page_content='卡、信用卡等)同時觸碰感應區,避免感應不良。  金屬物品會屏蔽或干擾 iQ System® **智慧鑰匙卡或** Smart Coin **的感應信號,使用時請確保卡片或** Smart Coin **與感應器之間沒有異物遮擋。** 若卡片或 Smart Coin **接觸感應器的時間太短或位置偏差太遠,可能會造成感應失敗,請將其拿**\n起、完全離開感應

In [158]:
main_db.similarity_search_with_score(query, k=5, filter={"car_model": "CROSSOVER"})

[(Document(page_content='左方視角\niQ System® 智慧鑰匙卡感應器位置\n「iQ System® 智慧鑰匙卡感應器」僅配備於使用無線鑰匙之車種。\n![22_image_0.png](app/static/CrossOver/22_image_0.png)', metadata={'#': 'Gogoro Crossover 系列 Smartscooter® **智慧電動機車** 使用手冊', '##': '2.1 Crossover 各部位名稱', 'page': 22, 'car_model': 'CROSSOVER'}),
  0.5630743),
 (Document(page_content='iQ System® 智慧鑰匙卡感應器的位置依車種不同,見下圖 :  ![37_image_0.png](app/static/CrossOver/37_image_0.png)  ![37_image_1.png](app/static/CrossOver/37_image_1.png)  ![37_image_2.png](app/static/CrossOver/37_image_2.png)  ![37_image_3.png](app/static/CrossOver/37_image_3.png)', metadata={'#': 'Gogoro Crossover 系列 Smartscooter® **智慧電動機車** 使用手冊', '##': '3. **準備上路**', '###': '3.2 **無線鑰匙車種**', '####': '3.2.2iQ System® 智慧鑰匙卡', '#####': '3.2.2.1 iQ System® 智慧鑰匙卡感應器位置', 'page': 38, 'car_model': 'CROSSOVER'}),
  0.65871334),
 (Document(page_content='卡、信用卡等)同時觸碰感應區,避免感應不良。  金屬物品會屏蔽或干擾 iQ System® **智慧鑰匙卡或** Smart Coin **的感應信號,使用時請確保卡片或** Smart Coin **與感應器之間沒有異物遮擋。** 若卡片或 Smart Coin **接觸感應器的時間太短或位